In [8]:
import numpy as np

In [1]:
import tensorflow as tf

In [19]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [20]:
print(x_train.shape)

(60000, 28, 28)


In [21]:
print(y_train.shape)

(60000,)


In [22]:
X_train = x_train.astype(np.float32).reshape(-1,28*28 ) / 255.0
X_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [23]:
print(X_train.shape)
print(y_train.shape)

(55000, 784)
(55000,)


In [24]:
n_inputs = 28*28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [26]:
#reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

In [27]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1]) # number of features as this is the second dimesnion of the dataset
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [28]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

In [29]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [30]:


learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)



In [31]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [32]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [33]:
n_epochs = 40
batch_size = 50
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch
        


with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

        

0 Batch accuracy: 0.94 Val accuracy: 0.9154
1 Batch accuracy: 0.96 Val accuracy: 0.9324
2 Batch accuracy: 0.96 Val accuracy: 0.9404
3 Batch accuracy: 0.98 Val accuracy: 0.948
4 Batch accuracy: 0.96 Val accuracy: 0.9512
5 Batch accuracy: 0.96 Val accuracy: 0.9536
6 Batch accuracy: 1.0 Val accuracy: 0.958
7 Batch accuracy: 0.98 Val accuracy: 0.9608
8 Batch accuracy: 0.98 Val accuracy: 0.9632
9 Batch accuracy: 0.96 Val accuracy: 0.9644
10 Batch accuracy: 0.98 Val accuracy: 0.9654
11 Batch accuracy: 0.98 Val accuracy: 0.9672
12 Batch accuracy: 1.0 Val accuracy: 0.967
13 Batch accuracy: 0.96 Val accuracy: 0.968
14 Batch accuracy: 1.0 Val accuracy: 0.9706
15 Batch accuracy: 1.0 Val accuracy: 0.9704
16 Batch accuracy: 0.98 Val accuracy: 0.9708
17 Batch accuracy: 1.0 Val accuracy: 0.973
18 Batch accuracy: 1.0 Val accuracy: 0.9722
19 Batch accuracy: 1.0 Val accuracy: 0.9738
20 Batch accuracy: 1.0 Val accuracy: 0.9734
21 Batch accuracy: 0.98 Val accuracy: 0.9742
22 Batch accuracy: 0.98 Val accur

In [34]:
import pandas as pd
test_f = pd.read_csv('test.csv')

In [45]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") # or better, use save_path
    test = test_f/255
    Z = logits.eval(feed_dict={X: test})
    y_pred = np.argmax(Z, axis=1)
    
    results = pd.Series(y_pred,name="Label")



INFO:tensorflow:Restoring parameters from ./my_model_final.ckpt


In [46]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("dnn_mnist_datagen.csv",index=False)